# Using Latent Class Analysis to explore (real) data

Now let's do the same stuff on real data.

In [ ]:
from lca import LCA
import pandas as pd

In [ ]:
from plotly import __version__ as plotly_version
from plotly.offline import init_notebook_mode, iplot

print("Plotly version: " + plotly_version)

init_notebook_mode(connected=True)         # initiate notebook for offline plot

# Introducing the data

We work with some data about which hobbies people have, from the "2003 INSEE survey on identity construction". The data concerns 8,403 individuals and includes Yes/No answers to "Which of the following leisure activities do you practice regularly" for 17 hobbies.

There are also some extra data: number of hours of TV per day on average, gender, age band, profession and marital status.

We also have a convenience column saying how many activities they answered yes to.

In [ ]:
hobby_cols = [
    'Reading',
    'ListeningToMusic',
    'Cinema',
    'Show',
    'Exhibition',
    'Computer',
    'Sport',
    'Walking',
    'Travelling',
    'PlayingMusic',
    'Collecting',
    'Volunteering',
    'Mechanic',
    'Gardening',
    'Knitting',
    'Cooking',
    'Fishing']

data_df = pd.read_csv("hobbies.csv", sep=";")

for col in hobby_cols:
    data_df[col] = data_df[col].apply(lambda x: 1 if x == 'y' else 0)
print('there are', len(data_df), 'rows')    
data_df.head(10)

I'm going to obtain rough age estimates by mapping each age band value to the central value:

In [ ]:
age_ests = {
    "[15,25]": 20,
    "(25,35]": 30,
    "(35,45]": 40,
    "(45,55]": 50,
    "(55,65]": 60,
    "(65,75]": 70,
    "(75,85]": 80,
    "(85,100]": 92.5
}

data_df['Age'] = data_df['AgeBand'].map(age_ests)

data_df.head(10)

In [ ]:
data_df.Profession.hist()

# What kinds of people are there?


Our question is: **What different kinds of people are there (as far as hobbies are concerned)?**
**Can we divide people into groups based on their hobbies?** This is an example of **unsupervised learning**.

The idea is that the population is composed of a number of different classes of people, and people from different classes tend to prefer different hobbies.

But we can't observe the classes of people directly (they are "Latent"); we can only infer their existence from the hobbies data.

Here I cut down to just people who answered with at least 7 and not more than 11 hobbies. This is a bit of ad-hocery motivated by:

 - People who tick nearly all the hobbies don't tell us much about how the hobbies are related, and nor do people with very few hobbies
 - I don't want the resulting classes to be "people who do lots of things" vs "people who don't do any things"

In [ ]:
#%matplotlib inline
#data_df.NumActivities.hist()

In [ ]:
print("originally there are", len(data_df), 'rows in the data frame')
data_df = data_df.loc[data_df['NumActivities'].isin([7, 8, 9, 10, 11])]
print("Cut down to data for %d people." % data_df.shape[0])

Let's get a matrix of just the hobbies responses.

In [ ]:
matrix = data_df[hobby_cols].values.copy()
matrix

The LCA routine will find the latent classes for us, but we need to (or, we get to) say how many we want.

In [ ]:
num_classes = 2

Let's do the business:

In [ ]:
lca = LCA(n_components=num_classes)
lca.fit(matrix)

print("Finished finding latent classes.")

Let's have a look at the resulting classes:

In [ ]:
chart_data = []

for i in range(num_classes):
    chart_data.append({
        'x': hobby_cols,
        'y': lca.theta[i, :],
        'type': 'bar',
        'name': "Class %d (%.1f%% of data)" % (i, lca.weight[i] * 100)
    })

figure = {
    'data': chart_data,
    'layout': {'yaxis': {'title': 'Probability of hobby'}}
}

iplot(figure)

A better way to visualise this is to order the columns by the ratio of the probabilities:

In [ ]:
probs_df = pd.DataFrame(hobby_cols, columns=['Hobby'])

for i in range(num_classes):
    probs_df['Class%dProb' % i] = lca.theta[i, :]

probs_df['Ratio'] = probs_df['Class0Prob'] / probs_df['Class1Prob']
probs_df = probs_df.sort_values('Ratio', ascending=False)
    
probs_df.head(10)
probs_df

In [ ]:
chart_data = []

for i in range(num_classes):
    chart_data.append({
        'x': probs_df['Hobby'],
        'y': probs_df['Class%dProb' % i],
        'type': 'bar',
        'name': "Class %d (%.1f%% of data)" % (i, lca.weight[i] * 100)
    })

figure = {
    'data': chart_data,
    'layout': {'yaxis': {'title': 'Probability of hobby'}}
}

iplot(figure)

**My interpretation:** one of our classes contains people who like "homely" activities like fishing, gardening, knitting, cooking, mechanic (working on their car, I guess). The other contains people who like more dynamic pursuits like going out to the cinema and shows, playing sport, going to exhibitions and travelling.

**Special note:** What I _particularly like_ about this approach is that the algorithm works out which hobbies are important indicators and which aren't. For hobbies where the two bars are nearly equal (e.g. ListeningToMusic or Reading) the algorithm has found that these hobbies aren't very informative about what other hobbies you have. But where there is a big difference in the bars, that hobby is informative. For example if you do Fishing, you are much more likely to do Gardening and much less likely to do Cinema. If you do Computer, you are much more likely to do Sport but much less likely to do Mechanic.

The next thing we get from the output of LCA is a prediction of which group each individual belows to. We can get both a "hard assignment" (each person placed crisply into a predicted class) or a "soft assignment" (where we get a predicted probability that the person is in each of the classes). We can add these back to the original data.

In [ ]:
data_df['PredictedClass'] = lca.predict(matrix)  # Hard assignment

soft_assignment = lca.predict_proba(matrix)

for i in range(num_classes):
    data_df['Class%dProb' % i] = soft_assignment[:,i]

data_df.head(10)

Because we have some additional data, we can cross-reference that against our clusters. Let's look at age for example.

In [ ]:
print("Mean age in class 0: %.1f" % data_df.loc[data_df['PredictedClass'] == 0]['Age'].mean())
print("Mean age in class 1: %.1f" % data_df.loc[data_df['PredictedClass'] == 1]['Age'].mean())

We can also look for differences in marital status:

In [ ]:
data_df.loc[data_df['PredictedClass'] == 0]['MaritalStatus'].value_counts(dropna=False, normalize=True)

In [ ]:
data_df.loc[data_df['PredictedClass'] == 1]['MaritalStatus'].value_counts(dropna=False, normalize=True)

In general we might need to do LCA in cases where we don't have any extra data, but in this case it helps us understand what the hobby-based clusters mean.

## Now let's go to more clusters! :)

This bit might take a few seconds to run:

In [ ]:
num_classes = 3

lca = LCA(n_components=num_classes)
lca.fit(matrix)

print("Finished finding latent classes.")

In [ ]:
data_df['PredictedClass'] = lca.predict(matrix)  # Hard assignment

soft_assignment = lca.predict_proba(matrix)

for i in range(num_classes):
    data_df['Class%dProb' % i] = soft_assignment[:,i]

data_df.head(10)

When graphing the columns we can't simply order by the ratio anymore, but we can approximately find the characteristic hobbies (or lack of hobbies) for each class by taking the ratio of the probability for the class with the average of the other two.

In [ ]:
def nice_prob_df_for_class(num_classes, lca, chosen_class):
    
    probs_df = pd.DataFrame(hobby_cols, columns=['Hobby'])

    for i in range(num_classes):
        probs_df['Class%dProb' % i] = lca.theta[i, :]

        
    # Find the average probability for all classes except the chosen class:
    
    other_class_probs = [probs_df['Class%dProb' % i] for i in range(num_classes) if i != chosen_class]

    total_prob_other_classes = other_class_probs[0]

    for col in other_class_probs[1:]:
        total_prob_other_classes = total_prob_other_classes + col  # Here we exploit the fact that you can add two Pandas series.
    
    probs_df['AvgOtherProb'] = total_prob_other_classes / (num_classes - 1)
    
    
    probs_df['Ratio'] = probs_df['Class%dProb' % chosen_class] / probs_df['AvgOtherProb']
    probs_df = probs_df.sort_values('Ratio', ascending=False)
    
    return probs_df

In [ ]:

for i in range(num_classes):
    probs_df = nice_prob_df_for_class(num_classes, lca, i)
    chart_data = []

    for i in range(num_classes):
        chart_data.append({
            'x': probs_df['Hobby'],
            'y': probs_df['Class%dProb' % i],
            'type': 'bar',
            'name': "Class %d (%.1f%%)" % (i, lca.weight[i] * 100)
        })

    figure = {
        'data': chart_data,
        'layout': {'yaxis': {'title': 'Probability of hobby'}}
    }

    iplot(figure)

**My interpretation:**

For three classes we have:

 - People who like Fishing, Mechanic and Gardening, do quite a lot of Sport and some Computer. They don't do Knitting, Show or Exhibition or Cinema. (One might guess that these are going to be older males.)
 - People who like Knitting, Cooking and Gardening, but don't do Fishing, Sport or Computer. (One might guess that these are older females.)
 - People who like Cinema, Show, Computer and Sport, but don't do Fishing, Knitting, Gardening, Mechanic or Cooking. (One might guess that these are younger people.)

**Let's have a look and see if I guessed right:**

In [ ]:
for i in range(num_classes):
    print("Mean age in class %d: %.1f" % (i, data_df.loc[data_df['PredictedClass'] == i]['Age'].mean()))

In [ ]:
print(pd.crosstab(data_df['PredictedClass'], data_df['Sex']))

In [ ]:
print(pd.crosstab(data_df['PredictedClass'], data_df['MaritalStatus']))

## Footnote

If we want to check convergence of the algorithm (that we ran it for enough iterations) we can do that as follows.
If the line has not flattened out, then you can add more iterations using the `max_iter` argument to `LCA`.

In [ ]:
figure = {
    'data': [
        {
           'x': list(range(1, len(lca.ll_))),
           'y': lca.ll_[1:]
        }],
    
    'layout': {
        'xaxis': { 'title': 'Iteration number' },
        'yaxis': { 'title': 'Log likelihood' }
        }
}

iplot(figure)

## Exercise 1

Apply the above approach to 4 clusters. For the clusters you get, try to arrive at a rough caricature of what somebody in that cluster looks like, based on the hobbies that people in that cluster typically do (and don't do), and what the data in the extra columns (hours of TV viewing, gender, age band, profession and marital status) looks like.


## Bonus Exercise (potentially time-consuming)

Run LCA on some different data. You could try for example:
 - Take the above data, and derive some extra attributes from the columns we didn't use (number of hours of TV per day, gender, age band, profession and marital status); see how putting these into the model rather than just analysing them afterwards changes the results.
 - Take a set of mention snippets (or news headlines) from a query and form Boolean attributes from the presence/absence of particular words.
 - Take some audiences data and use the interests or conversation topics as your attributes.
 - Take some data from a forum like MumsNet or Reddit and use the subforums as your attributes (with value 1 if a person posted on that subforum, and value 0 if they didn't).